In [2]:
import pandas as pd
import urllib3 as urllib
import json

In [ ]:
ls "C:/Users/MrSteve/Downloads/COVID_Vaccines_Shipped_LTC_712482_7 (1).xlsx"

In [10]:
df = pd.read_csv('C:/Users/MrSteve/Downloads/COVID-19_Nursing_Home_Dataset (2).tsv',  sep='\t')
df_latest = df[df['Week Ending'] == max(df['Week Ending'].unique())]

IL_DF = df[df["Provider State"] == "IL"]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
for col in df.columns:
    print(col)

Week Ending
Federal Provider Number
Provider Name
Provider Address
Provider City
Provider State
Provider Zip Code
Submitted Data
Passed Quality Assurance Check
Residents Weekly Admissions COVID-19
Residents Total Admissions COVID-19
Residents Weekly Confirmed COVID-19
Residents Total Confirmed COVID-19
Residents Weekly Suspected COVID-19
Residents Total Suspected COVID-19
Residents Weekly All Deaths
Residents Total All Deaths
Residents Weekly COVID-19 Deaths
Residents Total COVID-19 Deaths
Number of All Beds
Total Number of Occupied Beds
Resident Access to Testing in Facility
Laboratory Type Is State Health Dept
Laboratory Type Is Private Lab
Laboratory Type Is Other
Able to Test or Obtain Resources to Test All Current Residents Within Next 7 Days
Reason for Not Testing Residents - Lack of PPE for Personnel 
Reason for Not Testing Residents - Lack of Supplies
Reason for Not Testing Residents  - Lack of Access to Laboratory
Reason for Not Testing Residents - Lack of Access to Trained Pe

In [3]:
df_latest = df[df['Week Ending'] == max(df['Week Ending'].unique())]
#df_latest.to_csv('2020-05-31_CMD_DATA.csv')

In [4]:
df['confirmed-occupied_beds'] = df['Residents Total Confirmed COVID-19']/df_latest['Total Number of Occupied Beds']*100

In [5]:
cols = ['Week Ending', 'Federal Provider Number', 'Provider Name',
       'Provider Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'confirmed-occupied_beds', 
        'Residents Total Suspected COVID-19', 'Residents Total Admissions COVID-19',
        'Residents Total Confirmed COVID-19', 'Total Number of Occupied Beds', 
        'Number of All Beds', 'Total Number of Occupied Beds']
df[(df['confirmed-occupied_beds'] > 100.0) & df['Total Number of Occupied Beds'] > 0][cols].sort_values('confirmed-occupied_beds', ascending=False).to_csv('CMS_Data_More_Cases_Than_Beds.csv')

In [7]:
df.columns

Index(['Week Ending', 'Federal Provider Number', 'Provider Name',
       'Provider Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'Submitted Data', 'Passed Quality Assurance Check',
       'Residents Weekly Admissions COVID-19',
       'Residents Total Admissions COVID-19',
       'Residents Weekly Confirmed COVID-19',
       'Residents Total Confirmed COVID-19',
       'Residents Weekly Suspected COVID-19',
       'Residents Total Suspected COVID-19', 'Residents Weekly All Deaths',
       'Residents Total All Deaths', 'Residents Weekly COVID-19 Deaths',
       'Residents Total COVID-19 Deaths', 'Number of All Beds',
       'Total Number of Occupied Beds',
       'Resident Access to Testing in Facility',
       'Laboratory Type Is State Health Dept',
       'Laboratory Type Is Private Lab', 'Laboratory Type Is Other',
       'Staff Weekly Confirmed COVID-19', 'Staff Total Confirmed COVID-19',
       'Staff Weekly Suspected COVID-19', 'Staff Total Suspected COV

In [7]:
df_latest.columns

Index(['Week Ending', 'Federal Provider Number', 'Provider Name',
       'Provider Address', 'Provider City', 'Provider State',
       'Provider Zip Code', 'Submitted Data', 'Passed Quality Assurance Check',
       'Residents Weekly Admissions COVID-19',
       'Residents Total Admissions COVID-19',
       'Residents Weekly Confirmed COVID-19',
       'Residents Total Confirmed COVID-19',
       'Residents Weekly Suspected COVID-19',
       'Residents Total Suspected COVID-19', 'Residents Weekly All Deaths',
       'Residents Total All Deaths', 'Residents Weekly COVID-19 Deaths',
       'Residents Total COVID-19 Deaths', 'Number of All Beds',
       'Total Number of Occupied Beds',
       'Resident Access to Testing in Facility',
       'Laboratory Type Is State Health Dept',
       'Laboratory Type Is Private Lab', 'Laboratory Type Is Other',
       'Staff Weekly Confirmed COVID-19', 'Staff Total Confirmed COVID-19',
       'Staff Weekly Suspected COVID-19', 'Staff Total Suspected COV

In [3]:
#Filter by IL for states


IL_DF[IL_DF['Week Ending'] == '05/31/2020'].sum()[['Residents Total Confirmed COVID-19',  'Residents Total COVID-19 Deaths']]

Residents Total Confirmed COVID-19    7156
Residents Total COVID-19 Deaths       1961
dtype: object

In [4]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [5]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [6]:
df1 = pd.DataFrame(ltc_data['FacilityValues'])
df1['reporting_date'] = reporting_date
df1['% deaths'] = df1['deaths'] / df1['confirmed_cases']

In [7]:
df1

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths
0,Boone,Park Place of Belvidere,5,0,Open,2020-06-05,0.000000
1,Boone,Symphony Northwoods,87,13,Open,2020-06-05,0.149425
2,Champaign,Reflections Memory Care,8,0,Open,2020-06-05,0.000000
3,Champaign,University Rehab Center,5,0,Open,2020-06-05,0.000000
4,Christian,Villas of Holly Brook,2,0,Open,2020-06-05,0.000000
...,...,...,...,...,...,...,...
549,Winnebago,Peterson Meadows,7,1,Open,2020-06-05,0.142857
550,Winnebago,River Bluff Nursing Home,5,0,Open,2020-06-05,0.000000
551,Winnebago,Rock River Health Care,9,0,Open,2020-06-05,0.000000
552,Winnebago,Stepping Stone of Rockford,7,1,Open,2020-06-05,0.142857


In [8]:
f = open("../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json")

FileNotFoundError: [Errno 2] No such file or directory: '../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json'